Source of data -> https://www.londonair.org.uk/LondonAir/Default.aspx

Most of the functionality of this package is parsing the raw data which consists of nested dicts and returning data structures that are more convenient to work with.

API for hourly returns data in format (see `hourly.json`):
```
LocalAuthority (Borough)
    Site
        Species (CO2, NO2)
```

Not all Boroughs have sites, and generally each site might have a different combination of species.

In [1]:
# Required
from datetime import timedelta
import requests
from typing import List, Dict

# Optionals
import pandas as pd # not a dependency
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
CONF_LOCATIONS = "locations"

ALL_AUTHORITIES = [
     'Barking and Dagenham',
     'Barnet',
     'Bexley',
     'Brent',
     'Bromley',
     'Camden',
     'City of London',
     'Croydon',
     'Ealing',
     'Enfield',
     'Greenwich',
     'Hackney',
     'Hammersmith and Fulham',
     'Haringey',
     'Harrow',
     'Havering',
     'Hillingdon',
     'Hounslow',
     'Islington',
     'Kensington and Chelsea',
     'Kingston',
     'Lambeth',
     'Lewisham',
     'Merton',
     'Newham',
     'Redbridge',
     'Richmond',
     'Southwark',
     'Sutton',
     'Tower Hamlets',
     'Waltham Forest',
     'Wandsworth',
     'Westminster'
]

## Use only authorities with data, this might change over time
AUTHORITIES = [
    "Barking and Dagenham",
    "Bexley",
    "Brent",
    "Camden",
    "City of London",
    "Croydon",
    "Ealing",
    "Enfield",
    "Greenwich",
    "Hackney",
    "Haringey",
    "Harrow",
    "Havering",
    "Hillingdon",
    "Islington",
    "Kensington and Chelsea",
    "Kingston",
    "Lambeth",
    "Lewisham",
    "Merton",
    "Redbridge",
    "Richmond",
    "Southwark",
    "Sutton",
    "Tower Hamlets",
    "Wandsworth",
    "Westminster",
]

LAQ_HOURLY_URL = "http://api.erg.kcl.ac.uk/AirQuality/Hourly/MonitoringIndex/GroupName=London/Json"

TIMEOUT = 10

class LondonAirQualityException(Exception):
    pass

In [3]:
# AUTHORITIES = ALL_AUTHORITIES # Check functionality with all 

Helper for making the request

In [4]:
def request_data(url : str, timeout : int = TIMEOUT) -> Dict:
    """
    Request data from a URL and return valid data as dictionary.
    """
    try:
        response = requests.get(url, timeout=TIMEOUT)
        if response.status_code == 200:
            return response.json()
        else:
            raise LondonAirQualityException(
                f"Status code {response.status_code} returned from {url}")

    except requests.exceptions.Timeout:
        raise LondonAirQualityException(
            f"Request timeout, current timeout is {timeout} seconds"
        )

    except requests.exceptions.ConnectionError as exc:
        raise LondonAirQualityException(f"Internet connection error: {exc}")

In [5]:
def parse_hourly_response(hourly_response : Dict) -> Dict:
    """
    Return hourly response data to index by Borough.
    Allows filtering authorities with no data, and cleans up some data structure.
    """
    data = dict.fromkeys(AUTHORITIES)
    for authority in AUTHORITIES:
        try:
            for entry in hourly_response["HourlyAirQualityIndex"]["LocalAuthority"]:
                if entry["@LocalAuthorityName"] == authority:

                    if isinstance(entry["Site"], dict):
                        entry_sites_data = [entry["Site"]]
                    else:
                        entry_sites_data = entry["Site"]

                    data[authority] = parse_site(entry_sites_data)
        except Exception as exc: 
            data[authority] = {}
    return data

In [6]:
def parse_species(species_data : List[Dict]) -> List[Dict]:
    """Iterate over list of species at each site."""
    parsed_species_data = []
    quality_list = []
    for species in species_data:
        if species["@AirQualityBand"] != "No data":
            species_dict = {}
            species_dict["description"] = species["@SpeciesDescription"]
            species_dict["code"] = species["@SpeciesCode"]
            species_dict["quality"] = species["@AirQualityBand"]
            species_dict["index"] = species["@AirQualityIndex"]
            species_dict["summary"] = (
                species_dict["code"] + " is " + species_dict["quality"]
            )
            parsed_species_data.append(species_dict)
            quality_list.append(species_dict["quality"])
    return parsed_species_data, quality_list


def parse_site(entry_sites_data : List[Dict]) -> List[Dict]:
    """Iterate over all sites at an local authority and tidy the data."""
    authority_data = []
    for site in entry_sites_data:
        site_data = {}
        species_data = []

        site_data["updated"] = site["@BulletinDate"]
        site_data["latitude"] = site["@Latitude"]
        site_data["longitude"] = site["@Longitude"]
        site_data["site_code"] = site["@SiteCode"]
        site_data["site_name"] = site["@SiteName"].split("-")[-1].lstrip()
        site_data["site_type"] = site["@SiteType"]

        if isinstance(site["Species"], dict):
            species_data = [site["Species"]]
        else:
            species_data = site["Species"]

        parsed_species_data, quality_list = parse_species(species_data)

        if not parsed_species_data:
            parsed_species_data.append("no_species_data")
        site_data["pollutants"] = parsed_species_data

        if quality_list:
            site_data["pollutants_status"] = max(
                set(quality_list), key=quality_list.count
            )
            site_data["number_of_pollutants"] = len(quality_list)
        else:
            site_data["pollutants_status"] = "no_species_data"
            site_data["number_of_pollutants"] = 0

        authority_data.append(site_data)
    return authority_data

In [7]:
try:
    hourly_data_raw = request_data(LAQ_HOURLY_URL)
except LondonAirQualityException as exc:
    print(exc)

## Sites parsing
Parse out the sites data for a single local authority

In [8]:
example_site = hourly_data_raw['HourlyAirQualityIndex']['LocalAuthority'][0]['Site']
example_site

[{'@BulletinDate': '2019-09-26 09:00:00',
  '@SiteCode': 'BG1',
  '@SiteName': 'Barking and Dagenham - Rush Green',
  '@SiteType': 'Suburban',
  '@Latitude': '51.563752',
  '@Longitude': '0.177891',
  '@LatitudeWGS84': '6721627.34498',
  '@LongitudeWGS84': '19802.7355367',
  '@OwnerID': '1',
  'Species': [{'@SpeciesCode': 'NO2',
    '@SpeciesDescription': 'Nitrogen Dioxide',
    '@AirQualityIndex': '1',
    '@AirQualityBand': 'Low',
    '@IndexSource': 'Measurement'},
   {'@SpeciesCode': 'SO2',
    '@SpeciesDescription': 'Sulphur Dioxide',
    '@AirQualityIndex': '1',
    '@AirQualityBand': 'Low',
    '@IndexSource': 'Measurement'}]},
 {'@BulletinDate': '2019-09-26 09:00:00',
  '@SiteCode': 'BG2',
  '@SiteName': 'Barking and Dagenham - Scrattons Farm',
  '@SiteType': 'Suburban',
  '@Latitude': '51.529389',
  '@Longitude': '0.132857',
  '@LatitudeWGS84': '6715476.18683',
  '@LongitudeWGS84': '14789.5735883',
  '@OwnerID': '1',
  'Species': [{'@SpeciesCode': 'NO2',
    '@SpeciesDescripti

In [9]:
parse_site(example_site)

[{'updated': '2019-09-26 09:00:00',
  'latitude': '51.563752',
  'longitude': '0.177891',
  'site_code': 'BG1',
  'site_name': 'Rush Green',
  'site_type': 'Suburban',
  'pollutants': [{'description': 'Nitrogen Dioxide',
    'code': 'NO2',
    'quality': 'Low',
    'index': '1',
    'summary': 'NO2 is Low'},
   {'description': 'Sulphur Dioxide',
    'code': 'SO2',
    'quality': 'Low',
    'index': '1',
    'summary': 'SO2 is Low'}],
  'pollutants_status': 'Low',
  'number_of_pollutants': 2},
 {'updated': '2019-09-26 09:00:00',
  'latitude': '51.529389',
  'longitude': '0.132857',
  'site_code': 'BG2',
  'site_name': 'Scrattons Farm',
  'site_type': 'Suburban',
  'pollutants': [{'description': 'Nitrogen Dioxide',
    'code': 'NO2',
    'quality': 'Low',
    'index': '1',
    'summary': 'NO2 is Low'},
   {'description': 'PM10 Particulate',
    'code': 'PM10',
    'quality': 'Low',
    'index': '1',
    'summary': 'PM10 is Low'}],
  'pollutants_status': 'Low',
  'number_of_pollutants': 2

## Species parsing
The species parsing

In [10]:
example_species = example_site[0]['Species']
example_species

[{'@SpeciesCode': 'NO2',
  '@SpeciesDescription': 'Nitrogen Dioxide',
  '@AirQualityIndex': '1',
  '@AirQualityBand': 'Low',
  '@IndexSource': 'Measurement'},
 {'@SpeciesCode': 'SO2',
  '@SpeciesDescription': 'Sulphur Dioxide',
  '@AirQualityIndex': '1',
  '@AirQualityBand': 'Low',
  '@IndexSource': 'Measurement'}]

In [11]:
parse_species(example_species)

([{'description': 'Nitrogen Dioxide',
   'code': 'NO2',
   'quality': 'Low',
   'index': '1',
   'summary': 'NO2 is Low'},
  {'description': 'Sulphur Dioxide',
   'code': 'SO2',
   'quality': 'Low',
   'index': '1',
   'summary': 'SO2 is Low'}],
 ['Low', 'Low'])

## Package Usage

In [12]:
hourly_data = parse_hourly_response(hourly_data_raw)

In [13]:
hourly_data['Barking and Dagenham']

[{'updated': '2019-09-26 09:00:00',
  'latitude': '51.563752',
  'longitude': '0.177891',
  'site_code': 'BG1',
  'site_name': 'Rush Green',
  'site_type': 'Suburban',
  'pollutants': [{'description': 'Nitrogen Dioxide',
    'code': 'NO2',
    'quality': 'Low',
    'index': '1',
    'summary': 'NO2 is Low'},
   {'description': 'Sulphur Dioxide',
    'code': 'SO2',
    'quality': 'Low',
    'index': '1',
    'summary': 'SO2 is Low'}],
  'pollutants_status': 'Low',
  'number_of_pollutants': 2},
 {'updated': '2019-09-26 09:00:00',
  'latitude': '51.529389',
  'longitude': '0.132857',
  'site_code': 'BG2',
  'site_name': 'Scrattons Farm',
  'site_type': 'Suburban',
  'pollutants': [{'description': 'Nitrogen Dioxide',
    'code': 'NO2',
    'quality': 'Low',
    'index': '1',
    'summary': 'NO2 is Low'},
   {'description': 'PM10 Particulate',
    'code': 'PM10',
    'quality': 'Low',
    'index': '1',
    'summary': 'PM10 is Low'}],
  'pollutants_status': 'Low',
  'number_of_pollutants': 2

## Dataframe
We can also process the data into a list of dict that can be turned into a pandas dataframe (don't actually do this in the package as don't want pandas dependency)

In [14]:
def get_hourly_data_flat(hourly_data : Dict) -> List[Dict]:
    all_data = []
    for authority in hourly_data.keys():
        for site in hourly_data[authority]:        
            for pollutant in site['pollutants']:
                try:
                    pollutant['borough'] = authority
                    pollutant['site_code'] = site['site_code']
                    pollutant['site_name'] = site['site_name']
                    pollutant['latitude'] = site['latitude']
                    pollutant['longitude'] = site['longitude']
                    pollutant['updated'] = site['updated']
                    all_data.append(pollutant)
                except:
                    pass
    return all_data

In [15]:
df = pd.DataFrame(get_hourly_data_flat(hourly_data))

In [16]:
df.head()

,borough,code,description,index,latitude,longitude,quality,site_code,site_name,summary,updated
0,Barking and Dagenham,NO2,Nitrogen Dioxide,1,51.563752,0.177891,Low,BG1,Rush Green,NO2 is Low,2019-09-26 09:00:00
1,Barking and Dagenham,SO2,Sulphur Dioxide,1,51.563752,0.177891,Low,BG1,Rush Green,SO2 is Low,2019-09-26 09:00:00
2,Barking and Dagenham,NO2,Nitrogen Dioxide,1,51.529389,0.132857,Low,BG2,Scrattons Farm,NO2 is Low,2019-09-26 09:00:00
3,Barking and Dagenham,PM10,PM10 Particulate,1,51.529389,0.132857,Low,BG2,Scrattons Farm,PM10 is Low,2019-09-26 09:00:00
4,Bexley,NO2,Nitrogen Dioxide,1,51.4946486813055,0.137279111232178,Low,BQ7,Belvedere West,NO2 is Low,2019-09-26 09:00:00


In [17]:
boroughs_with_data = df['borough'].unique()
print(boroughs_with_data)
len(boroughs_with_data)

['Barking and Dagenham' 'Bexley' 'Brent' 'Camden' 'City of London'
 'Croydon' 'Ealing' 'Enfield' 'Greenwich' 'Hackney' 'Haringey' 'Harrow'
 'Havering' 'Hillingdon' 'Islington' 'Kensington and Chelsea' 'Kingston'
 'Lambeth' 'Lewisham' 'Merton' 'Redbridge' 'Richmond' 'Southwark' 'Sutton'
 'Tower Hamlets' 'Wandsworth' 'Westminster']


27

In [18]:
boroughs_without_data = list(set(ALL_AUTHORITIES) - set(boroughs_with_data))
boroughs_without_data

['Waltham Forest',
 'Barnet',
 'Hammersmith and Fulham',
 'Bromley',
 'Hounslow',
 'Newham']

In [19]:
df['code'].value_counts()

NO2     80
PM10    62
PM25    18
O3      17
SO2      5
Name: code, dtype: int64

In [20]:
df['quality'].value_counts()

Low    182
Name: quality, dtype: int64

In [21]:
df['index'].value_counts()

1    160
2     22
Name: index, dtype: int64